# Cours TAL – Labo 4
Nathan Gonzalez Montes et Vincent Guidoux

## Exercice 1 - exécuter la NER dans NLTK

### En utilisant nltk.ne_chunk (voir livre NLTK p. 283 - 4),extraire les entités nommées les plus fréquentes  d’un texte de votre choix

### En vous inspirant du POS de Stanford, utiliser leur NER (https://nlp.stanford.edu/software/CRF-NER.html) et la classe StanfordNERTagger de NLTK, pour extraire les entités nommées du même texte

### Comparez la liste des NE les plus fréquentes et leurs types reconnus

## Exercice 2
